# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Add paramter to disable streaming in the backend as it is unecessary for our gradio function.
- Choose the topic ✅

In [2]:
import sys
sys.path.append("..")

from src.gui import adversarial_chat
from src.test_chatbot import TestEris, TestSocrates

from IPython.display import Markdown, display
import gradio as gr
import time
import random

from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
# import torch

In [4]:
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

The system cannot find the path specified.


## Debate: Claude VS GPT

### Text-to-Speech

In [3]:
pipeline = KPipeline(lang_code='a', repo_id='hexgrad/Kokoro-82M')

d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [ ]:
# import soundfile as sf
# import tempfile
# import numpy as np
# from playsound import playsound

# def speak(text, voice="af_heart", save=False):
#     generator = pipeline(text, voice=voice)

#     for i, (_, _, audio) in enumerate(generator):
#         # Convierte el tensor a numpy
#         if hasattr(audio, "cpu"):
#             audio = audio.cpu().numpy()
        
#         if save:
#             sf.write(f"{i}.wav", audio, 24000)
#         else:
#             # Archivo temporal para reproducir
#             with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmpfile:
#                 sf.write(tmpfile.name, audio, 24000)
#                 playsound(tmpfile.name)


In [ ]:
def clean_message(message):
    """
    Replace double newlines with single newline.
    Useful because Claude like new paragraphs but we lack space.
    """
    return message.replace("\n\n", "\n").strip()




def speak(text, voice="af_heart"):
    generator = pipeline(text, voice=voice)
    for i, (_, _, audio_tensor) in enumerate(generator):
        audio_np = audio_tensor.numpy()  # convert to numpy array
        return (24000, audio_np)  # Gradio excepts a tuple (sample_rate, np.array)
    
    
def stream_message(history, message, is_socrates=True, delay=0.01):
    partial = ""
    for char in message:
        partial += char
        if is_socrates:
            if history and history[-1]["role"] == "user":
                history[-1]["content"] = partial
            else:
                history.append({"role": "user", "content": partial})
        else:
            if history and history[-1]["role"] == "assistant":
                history[-1]["content"] = partial
            else:
                history.append({"role": "assistant", "content": partial})
        time.sleep(delay)
        yield history, None  # emitimos sin audio por ahora

    # Cuando se completa el mensaje, generamos el audio
    audio = speak(partial)  # esto ya es (24000, np.array)
    return history, audio


In [ ]:
def update_history(history, message, is_socrates):
    if is_socrates:
        if history and history[-1]["role"] == "user":
            history[-1]["content"] = message
        else:
            history.append({"role": "user", "content": message})
    else:
        if history and history[-1]["role"] == "assistant":
            history[-1]["content"] = message
        else:
            history.append({"role": "assistant", "content": message})
    return history


def adversarial_chat(n_rounds, topic):
    socrates = TestSocrates()
    eris = TestEris()
    history = []

    time.sleep(random.uniform(0.25, 0.5))
    soc_reply = clean_message(socrates.ask(topic))
    
    partial = ""
    for char in soc_reply:
        partial += char
        history = update_history(history, partial, is_socrates=True)
        yield history, None

    audio = speak(soc_reply)
    yield history, audio  # return audio once message is fully spoken
    time.sleep(3)

    for i in range(n_rounds-1):
        time.sleep(random.uniform(0.25, 0.5))

        if i % 2 == 0:
            eris.history = socrates.history
            eris_reply = clean_message(eris.ask(soc_reply))
            partial = ""
            for char in eris_reply:
                partial += char
                history = update_history(history, partial, is_socrates=False)
                yield history, None
            audio = speak(eris_reply)
            yield history, audio
            time.sleep(3)
        else:
            socrates.history = eris.history
            soc_reply = clean_message(socrates.ask(eris_reply))
            partial = ""
            for char in soc_reply:
                partial += char
                history = update_history(history, partial, is_socrates=True)
                yield history, None
            audio = speak(soc_reply)
            yield history, audio
            time.sleep(3)



### GUI

In [12]:
# Gradio UI (edit only input and .click line)
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown("...")

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate")

    chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=200)
    # audio_output = gr.Audio(label="Current Speaker", autoplay=True)

    audio_output = gr.Audio(label="Current Speaker", autoplay=True, elem_id="audio_player")
    block_audio = gr.State(value=False)

    def reset_chat():
        return [], None

    start_btn.click(fn=reset_chat, outputs=[chatbox, audio_output])
    # start_btn.click(fn=adversarial_chat,
    #                 inputs=[rounds, topic_input],
    #                 outputs=[chatbox, audio_output])
    
    start_btn.click(
    fn=adversarial_chat,
    inputs=[rounds, topic_input],
    outputs=[chatbox, audio_output],
    js="""
        async (chat, audio) => {
            const player = document.getElementById("audio_player").querySelector("audio");
            if (player) {
                await new Promise((resolve) => {
                    player.onended = resolve;
                });
            }
            return [chat, audio];
        }
    """
)


demo.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\processing_utils.py:753: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


In [32]:
# demo.launch(share=True)